In [548]:
%load_ext autoreload
#To update the imported files

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [549]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [550]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet')
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')

In [551]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [552]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [553]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

In [554]:
#Cleaning data such that only wanted features are included

selected_features = ["date_forecast", "direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W",
"diffuse_rad_1h:J", "sun_elevation:d", "is_in_shadow:idx", "is_day:idx"]

#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

#Fjernen kvarter-radene og fjerner "date_calc" kolonnen hvis den finnes
X_train_observed_a_clean = clean_X_data(X_train_observed_a)
X_train_observed_b_clean = clean_X_data(X_train_observed_b)
X_train_observed_c_clean = clean_X_data(X_train_observed_c)
X_train_estimated_a_clean = clean_X_data(X_train_observed_a)
X_train_estimated_b_clean = clean_X_data(X_train_observed_b)
X_train_estimated_c_clean = clean_X_data(X_train_observed_c)
X_test_estimated_a_clean = clean_X_data(X_test_estimated_a)
X_test_estimated_b_clean = clean_X_data(X_test_estimated_b)
X_test_estimated_c_clean = clean_X_data(X_test_estimated_c)

X_train_observed_a_clean_selected_features = X_train_observed_a_clean[selected_features].copy()
X_train_observed_b_clean_selected_features = X_train_observed_b_clean[selected_features].copy()
X_train_observed_c_clean_selected_features = X_train_observed_c_clean[selected_features].copy()
X_train_estimated_a_clean_selected_features = X_train_estimated_a_clean[selected_features].copy()
X_train_estimated_b_clean_selected_features = X_train_estimated_b_clean[selected_features].copy()
X_train_estimated_c_clean_selected_features = X_train_estimated_c_clean[selected_features].copy()
X_test_estimated_a_clean_selected_features = X_test_estimated_a_clean[selected_features].copy()
X_test_estimated_b_clean_selected_features = X_test_estimated_b_clean[selected_features].copy()
X_test_estimated_c_clean_selected_features = X_test_estimated_c_clean[selected_features].copy()

In [555]:
"""
# Calculate time lapse since a reference time (e.g., the minimum date_forecast)
min_date_forecast = df['date_forecast'].min()
df['delta_time'] = (df['date_forecast'] - min_date_forecast).dt.total_seconds()

# Create cyclic month features
df['sine_month'] = np.sin((df['date_forecast'].dt.month - 1) * np.pi / 6)  # Assuming months are 1 to 12
df['cos_month'] = np.cos((df['date_forecast'].dt.month - 1) * np.pi / 6)

# Create cyclic hour features
min_hour_of_interest = 10
max_hour_of_interest = 15
df['delta_hour'] = df['date_forecast'].dt.hour - min_hour_of_interest
df['sine_hour'] = np.sin((df['delta_hour'] * np.pi / (max_hour_of_interest - min_hour_of_interest)))
df['cos_hour'] = np.cos((df['delta_hour'] * np.pi / (max_hour_of_interest - min_hour_of_interest)))
"""

"\n# Calculate time lapse since a reference time (e.g., the minimum date_forecast)\nmin_date_forecast = df['date_forecast'].min()\ndf['delta_time'] = (df['date_forecast'] - min_date_forecast).dt.total_seconds()\n\n# Create cyclic month features\ndf['sine_month'] = np.sin((df['date_forecast'].dt.month - 1) * np.pi / 6)  # Assuming months are 1 to 12\ndf['cos_month'] = np.cos((df['date_forecast'].dt.month - 1) * np.pi / 6)\n\n# Create cyclic hour features\nmin_hour_of_interest = 10\nmax_hour_of_interest = 15\ndf['delta_hour'] = df['date_forecast'].dt.hour - min_hour_of_interest\ndf['sine_hour'] = np.sin((df['delta_hour'] * np.pi / (max_hour_of_interest - min_hour_of_interest)))\ndf['cos_hour'] = np.cos((df['delta_hour'] * np.pi / (max_hour_of_interest - min_hour_of_interest)))\n"

In [556]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting
X_train_observed_a_clean_selected_features_resized, train_a_observed_resized = resize_trainingdata(X_train_observed_a_clean_selected_features, train_a, "date_forecast", y_features)
X_train_estimated_a_clean_selected_features_resized, train_a_estimated_resized = resize_trainingdata(X_train_estimated_a_clean_selected_features, train_a, "date_forecast", y_features)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_a_clean_selected_features_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_a_clean_selected_features_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, train_a_observed_resized["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(train_a_estimated_resized["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 245.43624911612486


In [562]:
X_train_estimated_a_clean_selected_features_resized

,direct_rad:W,direct_rad_1h:J,clear_sky_rad:W,clear_sky_energy_1h:J,diffuse_rad:W,diffuse_rad_1h:J,sun_elevation:d,is_in_shadow:idx,is_day:idx
0,0.0,0.0,0.0,0.000000,0.0,0.000000,-3.202000,1.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,-4.393000,1.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.000000,-3.910000,1.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,-1.986000,1.0,0.0
4,0.0,0.0,9.8,6546.899902,4.3,7743.299805,1.401000,0.0,1.0
...,...,...,...,...,...,...,...,...,...
29662,0.0,0.0,0.0,0.000000,0.0,0.000000,-27.830000,1.0,0.0
29663,0.0,0.0,0.0,0.000000,0.0,0.000000,-32.546001,1.0,0.0
29664,0.0,0.0,0.0,0.000000,0.0,0.000000,-35.674000,1.0,0.0
29665,0.0,0.0,0.0,0.000000,0.0,0.000000,-36.821999,1.0,0.0


In [558]:
#Training model, running infernece on testing data and saving it in a csv file ready for submission
X_train_observed_a_clean_selected_features_resized, train_a_observed_resized = resize_trainingdata(X_train_observed_a_clean_selected_features, train_a, "date_forecast", y_features)
X_train_estimated_a_clean_selected_features_resized, train_a_estimated_resized = resize_trainingdata(X_train_estimated_a_clean_selected_features, train_a, "date_forecast", y_features)

#Merging observed and estimated data to one big df
X_train_a = pd.concat([X_train_observed_a_clean_selected_features_resized, X_train_estimated_a_clean_selected_features_resized], ignore_index=True)
y_train_a = pd.concat([train_a_observed_resized, train_a_estimated_resized], ignore_index=True)
X_test = pd.concat([X_test_estimated_a_clean_selected_features, X_test_estimated_b_clean_selected_features, X_test_estimated_c_clean_selected_features], ignore_index=True)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_a_scaled = scaler.fit_transform(X_train_a)
X_test_a_scaled=scaler.fit_transform(X_test)

#Training the models
#Model A:
reg_a = LinearRegression()
reg_a.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions for A,B,C on the model trained on only A. Problems with the B and C datasets which need to be explored
y_pred = reg_a.predict(X_test_a_scaled)

#Saving y_pred in a proper csv file
filename = "CSV/second_submission.csv"
submission(filename, y_pred)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float32DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>, <class 'numpy.dtypes.Float32DType'>)

In [ ]:
%load_ext autoreload
#To update the imported files

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet')
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')

In [ ]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [ ]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [ ]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

In [ ]:
#Cleaning data such that only wanted features are included

selected_features = ["date_forecast", "direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W",
"diffuse_rad_1h:J", "sun_elevation:d", "is_in_shadow:idx", "is_day:idx"]

#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

#Fjernen kvarter-radene og fjerner "date_calc" kolonnen hvis den finnes
X_train_observed_a_clean = clean_X_data(X_train_observed_a)
X_train_observed_b_clean = clean_X_data(X_train_observed_b)
X_train_observed_c_clean = clean_X_data(X_train_observed_c)
X_train_estimated_a_clean = clean_X_data(X_train_observed_a)
X_train_estimated_b_clean = clean_X_data(X_train_observed_b)
X_train_estimated_c_clean = clean_X_data(X_train_observed_c)
X_test_estimated_a_clean = clean_X_data(X_test_estimated_a)
X_test_estimated_b_clean = clean_X_data(X_test_estimated_b)
X_test_estimated_c_clean = clean_X_data(X_test_estimated_c)

X_train_observed_a_clean_selected_features = X_train_observed_a_clean[selected_features].copy()
X_train_observed_b_clean_selected_features = X_train_observed_b_clean[selected_features].copy()
X_train_observed_c_clean_selected_features = X_train_observed_c_clean[selected_features].copy()
X_train_estimated_a_clean_selected_features = X_train_estimated_a_clean[selected_features].copy()
X_train_estimated_b_clean_selected_features = X_train_estimated_b_clean[selected_features].copy()
X_train_estimated_c_clean_selected_features = X_train_estimated_c_clean[selected_features].copy()
X_test_estimated_a_clean_selected_features = X_test_estimated_a_clean[selected_features].copy()
X_test_estimated_b_clean_selected_features = X_test_estimated_b_clean[selected_features].copy()
X_test_estimated_c_clean_selected_features = X_test_estimated_c_clean[selected_features].copy()

In [ ]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting
X_train_observed_a_clean_selected_features_resized, train_a_observed_resized = resize_trainingdata(X_train_observed_a_clean_selected_features, train_a, "date_forecast", y_features)
X_train_estimated_a_clean_selected_features_resized, train_a_estimated_resized = resize_trainingdata(X_train_estimated_a_clean_selected_features, train_a, "date_forecast", y_features)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_a_clean_selected_features_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_a_clean_selected_features_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, train_a_observed_resized["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(train_a_estimated_resized["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 245.43624911612486
